In [ ]:
from fastai.vision import *
import pandas as pd

In [ ]:
path = Path('../input')

In [ ]:
path.ls()

In [ ]:
Df= pd.read_csv(path/'train_labels.csv')
Df.head()

In [ ]:
Df['label'].unique()

In [ ]:
tfms = get_transforms(do_flip = True,flip_vert = True, max_rotate=15, max_zoom=1.15, max_lighting=0.4, max_warp=0.2,
                      p_affine=0.75, p_lighting=0.75)

In [ ]:
data = (ImageItemList.from_df(Df, path/'train', cols=0, folder = '.',suffix=".tif")
.random_split_by_pct(seed=42)
.label_from_df(cols=1)
.add_test_folder(test_folder = '../test')
.transform(tfms, size =96 )
.databunch(bs =64 )
.normalize(imagenet_stats))


In [ ]:
data

In [ ]:
from sklearn.metrics import roc_auc_score
def auc_score(y_pred,y_true,tens=True):
    score=roc_auc_score(y_true,torch.sigmoid(y_pred)[:,1])
    if tens:
        score=tensor(score)
    else:
        score=score
    return score

In [ ]:
learn = create_cnn(data, models.densenet121, path = '.', metrics =[accuracy, auc_score])

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, max_lr=slice(1e-06, 4e-04))

In [ ]:
learn.save('histo_stage2')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3,7e-06)

In [ ]:
learn.save('histo_stage3')

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(4e-06, 1e-04))

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(2,3e-05)

In [ ]:
preds,y=learn.get_preds()
pred_score=auc_score(preds,y)
pred_score

In [ ]:
preds,y=learn.TTA()
pred_score_tta=auc_score(preds,y)
pred_score_tta

In [ ]:
learn.save('histo_final')

In [ ]:
preds_test,y_test=learn.get_preds(ds_type=DatasetType.Test)
preds_test_tta,y_test_tta=learn.TTA(ds_type=DatasetType.Test)

In [ ]:
sub=pd.read_csv(path/'sample_submission.csv').set_index('id')
sub.head()

In [ ]:
clean_fname=np.vectorize(lambda fname: str(fname).split('/')[-1].split('.')[0])
fname_cleaned=clean_fname(data.test_ds.items)
fname_cleaned=fname_cleaned.astype(str)

In [ ]:
sub.loc[fname_cleaned,'label']=to_np(preds_test[:,1])
sub.to_csv(f'submission_{pred_score}.csv')

In [ ]:
sub.loc[fname_cleaned,'label']=to_np(preds_test_tta[:,1])
sub.to_csv(f'submission_{pred_score_tta}.csv')

In [ ]:
ls